In [25]:
import numpy as np
import cv2 as cv
import os
from copy import deepcopy

In [26]:
def show_image(title, image):
    image_resized = cv.resize(image, (0,0), fx=0.2, fy=0.2)
    cv.imshow(title, image_resized)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [27]:
def find_color_values_using_trackbar(frame):
    def nothing(x):
        pass

    cv.namedWindow('Trackbars')
    cv.createTrackbar('L - H', 'Trackbars', 0, 179, nothing)
    cv.createTrackbar('L - S', 'Trackbars', 0, 255, nothing)
    cv.createTrackbar('L - V', 'Trackbars', 0, 255, nothing)
    cv.createTrackbar('U - H', 'Trackbars', 179, 179, nothing)
    cv.createTrackbar('U - S', 'Trackbars', 255, 255, nothing)
    cv.createTrackbar('U - V', 'Trackbars', 255, 255, nothing)

    while True:
        l_h = cv.getTrackbarPos('L - H', 'Trackbars')
        l_s = cv.getTrackbarPos('L - S', 'Trackbars')
        l_v = cv.getTrackbarPos('L - V', 'Trackbars')
        u_h = cv.getTrackbarPos('U - H', 'Trackbars')
        u_s = cv.getTrackbarPos('U - S', 'Trackbars')
        u_v = cv.getTrackbarPos('U - V', 'Trackbars')

        l = np.array([l_h, l_s, l_v])
        u = np.array([u_h, u_s, u_v])

        frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        mask_table_hsv = cv.inRange(frame_hsv, l, u)

        res = cv.bitwise_and(frame, frame, mask=mask_table_hsv)

        scale_percent = 10
        width = int(frame.shape[1] * scale_percent / 100)
        height = int(frame.shape[0] * scale_percent / 100)
        dim = (width, height)

        resized_frame = cv.resize(frame, dim, interpolation=cv.INTER_AREA)
        resized_mask = cv.resize(mask_table_hsv, dim, interpolation=cv.INTER_AREA)
        resized_res = cv.resize(res, dim, interpolation=cv.INTER_AREA)

        cv.imshow("Frame", resized_frame)
        cv.imshow("Mask", resized_mask)
        cv.imshow("Result", resized_res)

        if cv.waitKey(1) & 0xFF == 27:
            break

    cv.destroyAllWindows()

In [28]:
# img = cv.imread("imagini_auxiliare/01.jpg")
# find_color_values_using_trackbar(img)

In [29]:
width = 2800
height = 2800
square_size = 200

In [30]:
def get_mask_table_hsv(img):
    # pt marginea tablei
	l = np.array([18, 128, 123])
	u = np.array([121, 255, 255])

	img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
	mask_hsv = cv.inRange(img_hsv, l, u)

	return mask_hsv

In [31]:
def get_full_board(img):
	mask_hsv = get_mask_table_hsv(img)
	img_copy = img.copy()
	contours, _ = cv.findContours(mask_hsv, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
	cv.drawContours(img_copy, contours, -1, (0, 255, 0), 10)
	for c in contours:
		area = cv.contourArea(c)
		x, y, w, h = cv.boundingRect(c)
		cv.rectangle(img_copy, (x, y), (x+w, y+h), (0, 255, 0), 10)

	max_area = 0

	for i in range(len(contours)):
		if len(contours[i]) > 3:
			possible_top_left = None
			possible_bottom_right = None
			for point in contours[i].squeeze():
				if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
					possible_top_left = point

				if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1]:
					possible_bottom_right = point

			diff = np.diff(contours[i].squeeze(), axis=1)	# perechi de y - x
			possible_top_right = contours[i].squeeze()[np.argmin(diff)]
			possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
			if cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]])) > max_area:
				max_area = cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]]))
				top_left = possible_top_left
				bottom_right = possible_bottom_right
				top_right = possible_top_right
				bottom_left = possible_bottom_left

	# cv.circle(img_copy, tuple(top_left), 20, (0, 0, 255), -1)
	# cv.circle(img_copy, tuple(top_right), 20, (0, 0, 255), -1)
	# cv.circle(img_copy, tuple(bottom_left), 20, (0, 0, 255), -1)
	# cv.circle(img_copy, tuple(bottom_right), 20, (0, 0, 255), -1)

	puzzle = np.array((top_left, top_right, bottom_right, bottom_left), dtype='float32')
	destination = np.array(([0, 0], [width, 0], [width, height], [0, height]), dtype='float32')
	M = cv.getPerspectiveTransform(puzzle, destination)
	result = cv.warpPerspective(img, M, (width, height), flags=cv.INTER_LINEAR)
	return result

In [32]:
def get_game_board(img):
    full_board = get_full_board(img)
    small_perct = 0.125
    big_perct = 0.875
    top_left = (small_perct * width, small_perct * width)
    top_right = (big_perct * width, small_perct * width)
    bottom_right = (big_perct * width, big_perct * width)
    bottom_left = (small_perct * width, big_perct * width)
    
    puzzle = np.array((top_left, top_right, bottom_right, bottom_left), dtype='float32')
    destination = np.array(([0, 0], [width, 0], [width, height], [0, height]), dtype='float32')
    M = cv.getPerspectiveTransform(puzzle, destination)
    
    board_copy = full_board.copy()
    board_copy = cv.warpPerspective(full_board, M, (width, height), flags=cv.INTER_LINEAR)
    return board_copy

In [33]:
empty_board_img = get_game_board(cv.imread("empty_board.jpg"))

In [34]:
empty_board = [
    ['x3', 'e', 'e', 'e', 'e', 'e', 'x3', 'x3', 'e', 'e', 'e', 'e', 'e', 'x3'],
    ['e', 'x2', 'e', 'e', '/', 'e', 'e', 'e', 'e', '/', 'e', 'e', 'x2', 'e'],
    ['e', 'e', 'x2', 'e', 'e', '-', 'e', 'e', '-', 'e', 'e', 'x2', 'e', 'e'],
    ['e', 'e', 'e', 'x2', 'e', 'e', '+', 'x', 'e', 'e', 'x2', 'e', 'e', 'e'],
    ['e', '/', 'e', 'e', 'x2', 'e', 'x', '+', 'e', 'x2', 'e', 'e', '/', 'e'],
    ['e', 'e', '-', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', '-', 'e', 'e'],
    ['x3', 'e', 'e', 'x', '+', 'e', '1', '2', 'e', 'x', '+', 'e', 'e', 'x3'],
    ['x3', 'e', 'e', '+', 'x', 'e', '3', '4', 'e', '+', 'x', 'e', 'e', 'x3'],
    ['e', 'e', '-', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', '-', 'e', 'e'],
    ['e', '/', 'e', 'e', 'x2', 'e', '+', 'x', 'e', 'x2', 'e', 'e', '/', 'e'],
    ['e', 'e', 'e', 'x2', 'e', 'e', 'x', '+', 'e', 'e', 'x2', 'e', 'e', 'e'],
    ['e', 'e', 'x2', 'e', 'e', '-', 'e', 'e', '-', 'e', 'e', 'x2', 'e', 'e'],
    ['e', 'x2', 'e', 'e', '/', 'e', 'e', 'e', 'e', '/', 'e', 'e', 'x2', 'e'],
    ['x3', 'e', 'e', 'e', 'e', 'e', 'x3', 'x3', 'e', 'e', 'e', 'e', 'e', 'x3']]

In [35]:
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
           '14', '15', '16', '17', '18', '19', '20', '21', '24', '25', '27', '28',
           '30', '32', '35', '36', '40', '42', '45', '48', '49', '50', '54', '56', 
           '60', '63', '64', '70', '72', '80', '81', '90']

In [36]:
def get_boards(folder):
    boards = []
    files = os.listdir(folder)
    for file in files:
        if file.endswith('.jpg'):
            img = cv.imread(folder + '/' + file)
            board = get_game_board(img)
            boards.append(board)
    return boards

In [37]:
def preprocess_image(img):
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_m_blur = cv.medianBlur(img_gray, 5)
    img_g_blur = cv.GaussianBlur(img_m_blur, (0, 0), 5)
    img_sharpened = cv.addWeighted(img_m_blur, 1.2, img_g_blur, -0.8, 0)
    _, thresh = cv.threshold(img_sharpened, 30, 255, cv.THRESH_BINARY)
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv.erode(thresh, kernel)
    return thresh

In [38]:
def preprocess_templates():
    os.makedirs("templates_processed", exist_ok=True)
    for i in range(46):
        img = cv.imread(f"templates/{i}.jpg")
        img = cv.resize(img, (150, 150))
        img = preprocess_image(img)
        cv.imwrite(f"templates_processed/{i}.jpg", img)

In [39]:
preprocess_templates()

In [40]:
def get_valid_posisions(board):
	valid_positions = []
	for i in range(14):
		for j in range(14):
			if board[i][j] in numbers:
				continue
			if i - 1 >= 0 and board[i-1][j] in numbers:
				valid_positions.append((i, j))
			elif i + 1 < 14 and board[i+1][j] in numbers:
				valid_positions.append((i, j))
			elif j - 1 >= 0 and board[i][j-1] in numbers:
				valid_positions.append((i, j))
			elif j + 1 < 14 and board[i][j+1] in numbers:
				valid_positions.append((i, j))
	return valid_positions

In [41]:
def locate_piece(prev_board, curr_board, game_board):
    diff = cv.absdiff(curr_board, prev_board)
    valid_positions = get_valid_posisions(game_board)
    max_diff, max_x, max_y = 0, 0, 0
    for x in range(0, 14):
        for y in range(0, 14):
            if (x, y) not in valid_positions:
                continue
            x_coord = x * square_size
            y_coord = y * square_size
            square = diff[x_coord:x_coord+square_size, y_coord:y_coord+square_size]
            if np.mean(square) > max_diff:
                max_diff = np.mean(square)
                max_x = x
                max_y = y
    return max_x, max_y

In [42]:
def classify_number(patch):
    patch = preprocess_image(patch)
    max_corr = -np.inf
    index = -1
    for j in range(0, 46):
        img_template = cv.imread('templates_processed/' + str(j) + '.jpg')
        if len(img_template.shape) == 3:
            img_template = cv.cvtColor(img_template, cv.COLOR_BGR2GRAY)
        corr = cv.matchTemplate(patch, img_template, cv.TM_CCOEFF_NORMED)
        corr = np.max(corr)
        if corr > max_corr:
            max_corr = corr
            index = j
    return numbers[index]

In [ ]:
def is_equation(num1, num2, result, operations):
    ok = False
    for op in operations:
        if op == '+' and num1 + num2 == result:
            ok = True
        elif op == '-' and (num1 - num2 == result or num2 - num1 == result):
            ok = True
        elif op == '*' and num1 * num2 == result:
            ok = True
        elif op == '/':
            if num2 != 0 and num1 / num2 == result:
                ok = True
            elif num1 != 0 and num2 / num1 == result:
            	ok = True
	return ok

In [44]:
def calculate_score(board, x, y, number):
    score = 0
    num_equations = 0
    if empty_board[x][y] in ['+', '-', '*', '/']:
        operations = [empty_board[x][y]]
    else:
        operations = ['+', '-', '*', '/']
    # left
    if y - 2 >= 0 and board[x][y-1] in numbers and board[x][y-2] in numbers:
        if is_equation(int(board[x][y-2]), int(board[x][y-1]), int(number), operations):
            num_equations += 1
    # right
    if y + 2 < 14 and board[x][y+1] in numbers and board[x][y+2] in numbers:
        if is_equation(int(board[x][y+1]), int(board[x][y+2]), int(number), operations):
            num_equations += 1
    # up
    if x - 2 >= 0 and board[x-1][y] in numbers and board[x-2][y] in numbers:
        if is_equation(int(board[x-2][y]), int(board[x-1][y]), int(number), operations):
            num_equations += 1
    # down
    if x + 2 < 14 and board[x+1][y] in numbers and board[x+2][y] in numbers:
        if is_equation(int(board[x+1][y]), int(board[x+2][y]), int(number), operations):
            num_equations += 1
            
    score = num_equations * int(number)
    if empty_board[x][y] == 'x2':
        score *= 2
    if empty_board[x][y] == 'x3':
        score *= 3
        
    return score

In [45]:
def save_scores(game_index, scores, turns, solution_folder):
	with open(solution_folder +  str(game_index) + "_scores.txt", "w") as f:
		for i in range(len(scores)):
			if i == len(scores) - 1:
				f.write(turns[i][0] + " " + str(turns[i][1]) + " " + str(scores[i]))
			else:
				f.write(turns[i][0] + " " + str(turns[i][1]) + " " + str(scores[i]) + "\n")

In [46]:
def solve(boards, solution_folder, data_folder):
    os.makedirs(solution_folder, exist_ok=True)
    num_boards = len(boards)
    num_games = num_boards // 50
    
    for i in range(num_games):
        with open(data_folder + str(i + 1) + "_turns.txt", "r") as f:
            turns = f.readlines()
            turns = [turn.strip().split() for turn in turns]
            turns = [(turn[0], int(turn[1])) for turn in turns]
            
        turn_index = 0
        end_turn = turns[turn_index + 1][1] - 1
        turn_score = 0
        scores = []
            
        game_board = deepcopy(empty_board)
        curr_boards = boards[i*50 : (i+1)*50]
        curr_boards.insert(0, empty_board_img)
        
        for j in range(1, 51):
            x, y = locate_piece(curr_boards[j-1], curr_boards[j], game_board)
            patch = curr_boards[j][max(x*square_size-50,0):min(x*square_size+square_size+50,height), max(y*square_size-50,0):min(y*square_size+square_size+50,width)].copy()
            number = classify_number(patch)
            game_board[x][y] = number
            score = calculate_score(game_board, x, y, number)
            turn_score += score
             
            if j < 10:
                with open(solution_folder + str(i + 1) + "_0" + str(j) + ".txt", "w") as f:
                    f.write(str(x + 1) + chr((y)%26+ord("A")) + " " + game_board[x][y])
            else:
                with open(solution_folder + str(i + 1) + "_" + str(j) + ".txt", "w") as f:
                    f.write(str(x + 1) + chr((y)%26+ord("A")) + " " + game_board[x][y])
                    
            if j == end_turn:
                scores.append(turn_score)
                turn_index += 1
                if turn_index < len(turns) - 1:
                    end_turn = turns[turn_index + 1][1] - 1
                else:
                    end_turn = 50
                turn_score = 0
                
        save_scores(i + 1, scores, turns, solution_folder)

In [48]:
data_folder = "evaluare/fake_test/"
solution_folder = "evaluare/fisiere_solutie/332_Hodivoianu_Anamaria/"

test_boards = get_boards(data_folder)
solve(test_boards, solution_folder, data_folder)